Den här koden är för att göra en del justeringar i koden så att det funkar med att läsa in .keras filer för spelning.  
Gör samtidigt om till Space Invaders.

In [49]:
import keras
from keras import layers
import gymnasium as gym
import ale_py
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing #stadnadrdiserar till 84 x 84 pixlar och gråskala
from gymnasium.wrappers.frame_stack import FrameStack # staplar 4 frames för att ge agenten en uppfattning om rörelse
import numpy as np
import tensorflow as tf

gym.register_envs(ale_py)

In [50]:
env = gym.make('SpaceInvadersNoFrameskip-v4', render_mode='rgb_array') #render_mode='rbg_array' för att få ut bilder
env = AtariPreprocessing(env)
env = FrameStack(env, num_stack=4)

num_actions = env.action_space.n #antal möjliga actions

#print(env.action_space.n)
#print(env.unwrapped.get_action_meanings())
# för att spara video
trigger = lambda t : t % 1
env = gym.wrappers.RecordVideo(env, video_folder = './space_videos', episode_trigger=trigger, disable_logger=True)

In [51]:
def create_q_model():
    return keras.Sequential(
        [
            layers.Lambda(
                lambda tensor: keras.ops.transpose(tensor, [0, 2, 3, 1]),
                output_shape=(84, 84, 4),
                input_shape=(4, 84, 84)
            ),
            layers.Conv2D(32, kernel_size=8, strides=4, activation="relu"),
            layers.Conv2D(64, kernel_size=4, strides=2, activation="relu"),
            layers.Conv2D(64, kernel_size=3, strides=1, activation="relu"),
            layers.Flatten(),
            layers.Dense(512, activation="relu"),
            layers.Dense(num_actions, activation="linear")
        ]
    )




In [52]:
model = create_q_model()
model_target = create_q_model()

optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

In [53]:
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

gamma = 0.99
epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 1.0
epsilon_interval = (
    epsilon_max - epsilon_min
)
batch_size = 32

# maximum episodes to run
max_episodes = 5 # 10000
# max frames to run
max_frames = 1e7
# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 1000000

# kolla förel
max_steps_per_episode = 10000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000
# Using huber loss for stability
loss_function = keras.losses.Huber()
# 

In [ ]:
while True:
    observation, _ = env.reset()
    state = np.array(observation)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # take random action
            action = env.action_space.sample()
        else:
            state_tensor = keras.ops.convert_to_tensor(state)
            state_tensor = keras.ops.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # Take best action
            action = keras.ops.argmax(action_probs[0].numpy())

        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        state_next, reward, done, _ , _ = env.step(action)
        state_next = np.array(state_next)
        episode_reward += reward

        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next
    
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            # get indices of smaples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)
            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            reward_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = keras.ops.convert_to_tensor([float(done_history[i]) for i in indices])

            future_rewards = model_target.predict(state_next_sample, verbose=0) # predikterar framtida rewards

            updated_q_values = reward_sample + gamma * keras.ops.amax(future_rewards, axis=1) # uppdaterar q-värden
            # if final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample
            # create a mask so we only calculate loss on the updated q values
            masks = keras.ops.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # training the model on the states and updated q-values
                q_values = model(state_sample)
                q_action = keras.ops.sum(keras.ops.multiply(q_values, masks), axis=1)
                loss = loss_function(updated_q_values, q_action)
                # backpropagation:
                grads = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            model_target.set_weights(model.get_weights())
            # Log details
            print(f"best score of last 100 episodes: {np.max(episode_reward_history)}, running_reward: {running_reward}, at episode {episode_count} frame_count: {frame_count}")       
            model.save(f"breakout_model_{episode_count}.keras")

        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > 300:
        print(f"Solver at epiode{episode_count}")
        model.save(f"breakout_model_{episode_count}.keras")
        break
    if (max_episodes > 0 and episode_count >= max_episodes):
        print(f"stopped at episode {episode_count}")
        break
    if (max_frames > 0 and frame_count >= max_frames):
        print(f"stopped at frame {frame_count}")
        break
        

Solver at epiode1
